In [1]:
'''
Created on 2018. 2. 14.

@author: phs

1. 대화 말뭉치 파일을 읽어들인다.
2. 딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어 들여서 반환한다.
3. 딥러닝(tensorflow)으로 생성한 자연어 이해 모델을 읽어들여서 반환한다.
4. 입력받은 문장을 자연어 처리한다(토크나이저, 스태밍).
5. 자연어 처리한 문장을 Bag of word 생성하여 반환한다.
6. 입력받은 문장을 학습 모델(자연어 이해 모델)을 이용 분류하여  결과를 반환한다.
7. 분류된 말뭉치 대화에서 임의로 한 문장을 선택하여 입력받은 문장의 대답으로 반혼한다.

'''
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# things we need for NLP
import json
import pickle
import random

# import nltk
# from nltk.stem.lancaster import LancasterStemmer
# stemmer = LancasterStemmer()

# from konlpy.tag import Komoran
from konlpy.tag import Okt
# komoran = Komoran()
twitter = Okt()


import tensorflow as tf
import numpy as np


In [2]:
# import our chat-bot intents file
def read_dialog_intents_jsonfile(input_file_name):
    """
     대화 말뭉치 파일을 읽어들인다.
    """
    with open(input_file_name, 'rt', encoding='UTF8') as json_data:
        intents = json.load(json_data)
        
    return intents

# 대화 말뭉치와 대화 의도가 정의된 JSON 문서 집합 읽기
input_file_name = './DialogIntents/intents_home_kr.json'
intents = read_dialog_intents_jsonfile(input_file_name)


In [3]:
# restore all of our data structures
def restore_training_data_structures(input_training_data_file_name):
    """
     딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어 들여서 반환한다.
    """
    # restore all of our data structures
    data = pickle.load( open( input_training_data_file_name, "rb" ) )
    words = data['words']
    classes = data['classes']
    train_x = data['train_x']
    train_y = data['train_y']
    
    return classes, words, train_x, train_y

#  딥러닝(tensorflow) 모델 생성에 사용한 자료구조를 읽어들임
input_training_data_file_name = "./NLUModel/training_data_home_kr_tf2"
classes, words, train_x, train_y = restore_training_data_structures(input_training_data_file_name)


In [4]:
print (len(train_x), "개의 train_x")
print (train_x[0], "data of train_x[0]")
print (len(train_y), "개의 train_y")
print (train_y[0], "data of train_y[0]")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

54 개의 train_x
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] data of train_x[0]
54 개의 train_y
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] data of train_y[0]
11 classes ['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
77 unique stemmed words ['.', '가', '감사', '감사하다', '개발', '개선', '거기', '경력', '경험', '계세', '계시다', '고맙다', '과', '그렇다', '꺼지다', '나중', '날', '누구', '는', '다음', '닥치다', '돈', '들리다', '또', '를', '많이', '멍청이', '무슨', '바보', '버세', '번', '보완', '부분', '부족', '부탁', '사라지다', '사항', '새끼', '서비스', '소개', '수고', '안녕', '안녕하다', '야', '약점', '어떻다', '어학', '언제', '업적', '에', '에요', '여기', '영어', '요', '은', '을', '이', '이다', '일본어', '임마', '있다', '자기', '자다', '자신', '장사', '장점', '전공', '제품', '졸업', '좋다', '줄다', '중국어', '지내다', '친절하다', '하다', '학교', '해주다']


In [5]:
# load our saved model
tf2_model_file_name = './NLUModel/model_home_kr_tf2'
model = tf.keras.models.load_model(tf2_model_file_name)


In [6]:
# things we need for Tensorflow

# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.70

def clean_up_sentence(sentence):
    # tokenize the pattern
#     sentence_words = nltk.word_tokenize(sentence)
    pos_result = twitter.pos(sentence, norm=True, stem=True)
#     print("sentence pos_result[%s]" % pos_result)
#     print("sentence pos_result[0][0][%s]" % pos_result[0][0])
#     print("sentence pos_result[1][0][%s]" % pos_result[1][0])
#     print("sentence pos_result[][%s]" % pos_result[0])
    sentence_words = [lex for lex, pos in pos_result]
    # stem each word
#     sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

#     return(np.array(bag))
    return bag


def classify(sentence, words, classes, model):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, intents, words, classes, model, userID='123', show_details=False):
    results = classify(sentence, words, classes, model)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return random.choice(i['responses'])

            results.pop(0)


In [7]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['안녕',
    '안녕하세요',
    '잘 지냈어요?',
    '여기요',
    '잘 계셨어요?',
    '거기 누구 있어요?',
    '계세요',
    '계셔요',
    '좋은 날 입니다'],
   'responses': ['안녕하세요, 방문해 주셔서 감사합니다.',
    '다시 만나서 반가워요.',
    '만나서 반갑습니다',
    '찾아 주셔서 감사합니다',
    '안녕하세요, 어떻게 도와 드릴까요?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['안녕히 계셔요.',
    '그럼 나중에 또 들릴께요',
    '나중에요',
    '다음에',
    '다음번에',
    '장사 잘 하세요',
    '돈 많이 버세요',
    '수고하세요',
    '친절한 서비스에 감사합니다.'],
   'responses': ['나중에 또 오세요, 방문해 주셔서 감사합니다.',
    '좋은 하루 되세요.',
    '안녕히 가세요',
    '감사합니다.',
    '고맙습니다',
    '즐거운 시간 되세요']},
  {'tag': 'thanks',
   'patterns': ['감사', '감사합니다', '고맙습니다', '좋은 서비스 감사합니다'],
   'responses': ['감사합니다']},
  {'tag': 'Slang',
   'patterns': ['멍청이', '바보', '야임마', '꺼져', '사라져', '새끼', '닥쳐'],
   'responses': ['좋은 말로 해주세요.',
    '듣기가 거북하네요',
    '앞으로 더 노력 하겠습니다.',
    '좋은 모습으로 만나 뵐 수 있기를 바랍니다.']},
  {'tag': '자기소개',
   'patterns': ['자기자신 소개를 해주세요?', '자기자신 소개를 부탁합니다?'],
   'responses': ['저는

In [8]:
sentence = '안녕?'
print("classify('안녕?')[{}]".format(classify(sentence, words, classes, model)))
print("response('안녕?') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()


classify('안녕?')[[('greeting', 0.9997557)]]
response('안녕?') ==> [다시 만나서 반가워요.]



In [9]:
context

{'123': ''}

In [10]:
sentence = '자기자신 소개를 부탁합니다?'
print("classify('자기자신 소개를 부탁합니다?')[%s]" % classify(sentence, words, classes, model))
print("response('자기자신 소개를 부탁합니다?') ==> [%s]" % response(sentence, intents, words, classes, model))
print()

classify('자기자신 소개를 부탁합니다?')[[('자기소개', 0.9998404)]]
response('자기자신 소개를 부탁합니다?') ==> [저는 서울에 사는 홍길동이라고 합니다. 캐글 대학에서 자연어 처리를 전공하였고 재학중 자연어처리 기반 테이터사이언스 경진 대회에서 참가하여 수상 실적이 있습니다]



In [11]:
# show context
context

{'123': ''}

In [12]:
sentence = '보완 사항은?'
print("classify('보완 사항은?')[%s]" % classify(sentence, words, classes, model))
print("response('보완 사항은?') ==> [%s]" % response(sentence, intents, words, classes, model))
print()

classify('보완 사항은?')[[('약점', 0.9999044)]]
response('보완 사항은?') ==> [일을 너무 신중하게 하는것입니다.]



In [13]:
# show context
context

{'123': 'serviceTechnology'}

In [14]:
sentence = '잘 하는것은?'
print("classify('잘 하는것은?')[{}]".format(classify(sentence, words, classes, model)))
print("response('잘 하는것은?') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('잘 하는것은?')[[('장점', 0.9981375)]]
response('잘 하는것은?') ==> [건강하고 설실합니다]



In [15]:
# clear context
#response("Hi there!", show_details=True)
sentence = '안녕'
print("response('안녕?') ==> [{}]".format(response(sentence, intents, words, classes, model,show_details=True)))
print("classify('안녕?')[{}]".format(classify(sentence, words, classes, model)))
print()

context: 
tag: greeting
response('안녕?') ==> [찾아 주셔서 감사합니다]
classify('안녕?')[[('greeting', 0.9997557)]]



In [16]:
sentence = '학교졸업'
print("classify('학교졸업')[{}]".format(classify(sentence, words, classes, model)))
print("response('학교졸업') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('학교졸업')[[('학력', 0.99988914)]]
response('학교졸업') ==> [캐글대학 빅데이터처리학과를 2020년 2월에 졸업하였습니다.]



In [17]:
# show context
context

{'123': 'serviceTechnology'}

In [18]:
sentence = '안녕히 계셔요.'
print("classify('안녕히 계셔요.')[{}]".format(classify(sentence, words, classes, model)))
print("response('안녕히 계셔요.') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('안녕히 계셔요.')[[('goodbye', 0.9997837)]]
response('안녕히 계셔요.') ==> [즐거운 시간 되세요]



In [19]:
# show context
context

{'123': 'serviceTechnology'}

In [20]:
sentence = '뭐야?'
print("classify('뭐야?')[{}]".format(classify(sentence, words, classes, model)))
print("response('뭐야?') ==> [{}]".format(response(sentence, intents, words, classes, model)))
print()

classify('뭐야?')[[('Slang', 0.9894112)]]
response('뭐야?') ==> [앞으로 더 노력 하겠습니다.]

